In [1]:
import site
site.getsitepackages()

['c:\\Users\\Francisco Azeredo\\.conda\\envs\\tese',
 'c:\\Users\\Francisco Azeredo\\.conda\\envs\\tese\\Lib\\site-packages']

In [2]:
import weaviate

In [11]:
client = weaviate.connect_to_local()
print(client.is_connected())

c:\Users\Francisco Azeredo\.conda\envs\tese\Lib\site-packages\weaviate\warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
C:\Users\Francisco Azeredo\AppData\Local\Temp\ipykernel_41956\3746343668.py:1: ResourceWarning: unclosed <socket.socket fd=1980, family=23, type=1, proto=0, laddr=('::1', 50861, 0, 0), raddr=('::1', 8080, 0, 0)>
  client = weaviate.connect_to_local()


True


In [21]:
response = client.collections.list_all(simple=True)

print(response)

{'Entidade': _CollectionConfigSimple(name='Entidade', description='An entity that owns folders (pastas).', generative_config=_GenerativeConfig(generative=<GenerativeSearches.OLLAMA: 'generative-ollama'>, model={'apiEndpoint': 'http://host.docker.internal:11434', 'model': 'qwen2.5:latest'}), properties=[_Property(name='name', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={'text2vec-transformers': _PropertyVectorizerConfig(skip=False, vectorize_property_name=False)})], references=[_ReferenceProperty(name='hasFicheiros', description=None, target_collections=['Ficheiro']), _ReferenceProperty(name='hasPastas', description=None, target_collections=['Pasta'])], reranker_config=None, vectorizer_config=None, vectorizer=None, vector_config={'text_vector': _NamedVectorConfig(vectorizer=_NamedVe

In [26]:
document = client.collections.get("Fluxo")
document

In [ ]:
client.collections.delete_all()

In [7]:
client.collections.delete("Dataset")  # THIS WILL DELETE THE SPECIFIED COLLECTION(S) AND THEIR OBJECTS

Collection creation

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

api_endpoint = "http://host.docker.internal:11434"
# api_endpoint = "http://localhost:11434"  # If not using Docker, use this to contact your local Ollama instance
client.collections.create(
    "DemoCollection",
    properties=[
        Property(name="page", data_type=DataType.INT),
        Property(name="text", data_type=DataType.TEXT)
    ],
    # vectorizer_config=Configure.Vectorizer.text2vec_ollama(),
    # api_endpoint="http://host.docker.internal:11434", in case running in linux
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["page", "text"],
            api_endpoint=api_endpoint,
            model="mxbai-embed-large"
        ),
        Configure.NamedVectors.text2vec_huggingface(
            name="text_vector",
            source_properties=["text"],
            model="sentence-transformers/all-MiniLM-L6-v2"
        ),
    ],
    generative_config=Configure.Generative.ollama(
        api_endpoint=api_endpoint,  # If using Docker, use this to contact your local Ollama instance
        model="llama3.2"  # The model to use, e.g. "phi3", or "mistral", "command-r-plus", "gemma"
    )
    # Additional parameters not shown
)

Data Import

In [40]:
collection = client.collections.get("DemoCollection")

with open("output.txt", "r", encoding="utf-8") as f:
    page_texts = f.read().strip().split("---")

# ✅ Open batch context once (outside the loop)
with collection.batch.dynamic() as batch:
    for idx, text in enumerate(page_texts):
        batch.add_object(
            properties={"page": idx + 1, "text": text.strip()}  # ✅ Correct parameter name
        )
        batch.flush()  # ✅ Flush after each object


In [41]:
print(collection.batch.failed_objects)
result = collection.query.fetch_objects(limit=10)  # Adjust limit as needed

for obj in result.objects:
    print(f"Page {obj.properties['page']}: {obj.properties['text'][:200]}...")  # Show first 200 chars


[ErrorObject(message='WeaviateInsertManyAllFailedError(\'Every object failed during insertion. Here is the set of all errors: send POST request: Post "http://localhost:11434/api/embeddings": dial tcp [::1]:11434: connect: connection refused\')', object_=BatchObject(collection='DemoCollection', properties={'page': 1, 'text': 'Page 1:\n\n\n**Keyphrase Extraction and Geospatial Characterizations for the Usage of Keyphrases**\n\n**Francisco Jose Guerra Carreira**\n\nThesis to obtain the Master of Science Degree in\n\n**Information Systems and Computer Engineering**\n\nSupervisors: Prof. Bruno Emanuel da Graca Martins\n\nProf. Miguel Daiyen Carvalho Won\n\n**Examination Committee**\n\nChairperson: Prof. Ana Maria Severino de Almeida e Paiva\n\nSupervisor: Prof. Bruno Emanuel da Graca Martins\n\nMember of the Committee: Prof. Ricardo Daniel Santos Faro Marques Ribeiro\n\n**October 2017**'}, references=None, uuid='b81ad333-4d55-4e51-9ba7-39ddc02239fb', vector=None, tenant=None, index=0, retry

Query by text search

In [ ]:
query_text = "keyphrase extraction"

result = collection.query.near_text(
    query=query_text,  # ✅ Use query inside near_text
    limit=5  # ✅ Limit to 5 results
)

if result.objects:
    for obj in result.objects:
        print(f"Page {obj.properties['page']}: {obj.properties['text']}...")
else:
    print("No relevant results found.")

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "explorer: get class: concurrentTargetVectorSearch): explorer: get class: vectorize search vector: vectorize params: vectorize params: vectorize keywords: remote client vectorize: send POST request: Post "http://localhost:11434/api/embeddings": dial tcp [::1]:11434: connect: connection refused"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"explorer: get class: concurrentTargetVectorSearch): explorer: get class: vectorize search vector: vectorize params: vectorize params: vectorize keywords: remote client vectorize: send POST request: Post \"http://localhost:11434/api/embeddings\": dial tcp [::1]:11434: connect: connection refused", grpc_status:2, created_time:"2025-02-21T15:44:09.9992493+00:00"}"
>.

In [ ]:
client.close()